# Feature_Engineering

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("train.csv")

'property_type'  sutununu 5 gruba ayirmis

In [ ]:
dict1 = {'Apartment':['Condominium','Timeshare','Loft','Serviced apartment','Guest suite'],
         'House':['Vacation home','Villa','Townhouse','In-law','Casa particular'],
         'Hotel1':['Dorm','Hostel','Guesthouse'],
         'Hotel2':['Boutique hotel','Bed & Breakfast'],
         'Other':['Island','Castle','Yurt','Hut','Chalet','Treehouse',
                  'Earth House','Tipi','Cave','Train','Parking Space','Lighthouse',
                 'Tent','Boat','Cabin','Camper/RV','Bungalow']
        }
dict2 = {i : k for k, v in dict1.items() for i in v}
data['property_type'].replace(dict2).value_counts()

baska bir yol

In [ ]:
# Replacing categories that are types of houses or apartments
df.property_type.replace({
    'Townhouse': 'House',
    'Serviced apartment': 'Apartment',
    'Loft': 'Apartment',
    'Bungalow': 'House',
    'Cottage': 'House',
    'Villa': 'House',
    'Tiny house': 'House',
    'Earth house': 'House',
    'Chalet': 'House'  
    }, inplace=True)

# Replacing other categories with 'other'
df.loc[~df.property_type.isin(['House', 'Apartment']), 'property_type'] = 'Other'

mahalle sayisini 619 bulmus,city ye göre gruplamis ve mahalleyi drop etme karari almis

city  neighbourhood	
Boston	34
Chicago	80
DC	    116
LA	    157
NYC	   201
SF	   55

In [ ]:
data['neighbourhood'].nunique()

pd.DataFrame(data.groupby(['city'])['neighbourhood'].nunique())

one hat encoding yapacagi sutunlari bu sekilde secmis

In [ ]:
#columns for get_dummies (one hot encoding)
col_to_getdummies=['property_type','room_type','bed_type','cancellation_policy','city']

#columns that won't be changed
col_no_change=['accommodates','number_of_reviews']

room type 3 ,2,1  diye numaralandirmis!

In [ ]:
cleaned_data.room_type = cleaned_data.room_type.apply(lambda x: 3 if x=='Entire home/apt' else 2 if x=='Private room' else 1)

bed_type 2,1 olarak numaralandirmis!

Real Bed         78656
Pull-out Sofa      448
Futon              195
Couch               74
Airbed              50

Most listings have the same bed type so this feature can be dropped

verinin cogu ayni old icin drop etmis 

In [ ]:
df.bed_type.value_counts()
df.drop('bed_type', axis=1, inplace=True)

In [ ]:
cleaned_data.bed_type = cleaned_data.bed_type.apply(lambda x: 2 if x=='Real Bed' else 1)

cancellation_policy  3,2,1 olarak numaralandirmis

In [ ]:
cleaned_data.cancellation_policy = cleaned_data.cancellation_policy.apply(lambda x: 3 if x=='super_strict_60' else 2 if x=='super_strict_30' else 1)

future larin etkisini görmek icin...

In [ ]:
plot_importance(model)

ilk kayit oldugu tarihten gunumuze geen zamana göre kiyaslama yapabilmeyi bu sekilde yapmis

In [ ]:
# Converting to datetime
df.host_since = pd.to_datetime(df.host_since) 

# Calculating the number of days
df['host_days_active'] = (datetime(2019, 4, 9) - df.host_since).astype('timedelta64[D]')

# Printing mean and median
print("Mean days as host:", round(df['host_days_active'].mean(),0))
print("Median days as host:", df['host_days_active'].median())

# Replacing null values with the median
df.host_days_active.fillna(df.host_days_active.median(), inplace=True)

Bu işlemlerle df.host_response_rate sütunu kategorilere ayrılmış, eksik değerler 'unknown' olarak işaretlenmiş ve her bir kategoriye ait değerlerin sayıları hesaplanmıştır.

100%       37730
unknown    25699
50-89%      7346
90-99%      6519
0-49%       2377

bu sonuc alinmis

In [ ]:
# Bin into four categories
df.host_response_rate = pd.cut(df.host_response_rate, bins=[0, 50, 90, 99, 100], labels=['0-49%', '50-89%', '90-99%', '100%'], include_lowest=True)

# Converting to string
df.host_response_rate = df.host_response_rate.astype('str')

# Replace nulls with 'unknown'
df.host_response_rate.replace('nan', 'unknown', inplace=True)

# Category counts
df.host_response_rate.value_counts()